In [ ]:
import yfinance as yf
import pandas as pd
import pandas_datareader as pdr
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import warnings
from scipy.stats import norm, jarque_bera

def filtrage_ETF(etf_ticker_placeholder):
    # Placeholder tickers
    placeholder_tickers = list(set([
        'NVDA','AAPL','MSFT','AMZN','GOOGL','GOOG','META','AVGO','TSLA','LLY','JPM','UNH','XOM','V','MA','COST','ORCL','HD','PG',
        'NFLX','ABBV','BAC','SAP','ASML','CRM','WMT','KO','CVX','TM','ADBE','PEP','MRK','LIN','TMO','WFC','MCD','ACN','CSCO','ABT',
        'DIS','QCOM','GE','INTU','DHR','CAT','VZ','TXN','IBM','AMAT','AXP','PFE','MS','PM','AMGN','GS','ISRG','NOW','HON','RTX','LOW',
        'BKNG','SPGI','INTC','SYK','C','TJX','PGR','BLK','ELV','REGN','SCHW','VRTX','ETN','BSX','LMT','ADI','PANW','MDLZ','CB','CI',
        'MMC','BA','MU','PLTR','GILD','ADP','LRCX','DE','VRT','T','WM','AMT','MDT','ZTS','ICE','SBUX','MO','ITW','SHW','ANET','CVS',
        'HCA','BX','EQIX','ECL','PH','GD','MCK','ORLY','CDNS','MAR','NXPI','CTAS','BDX','APH','MCO','TGT','F','EMR','ADSK','ROP','TDG',
        'KLAC','AON','CMG','PNC','TEL','MET','CARR','SNPS','EW','DASH','APO','DLR','AJG','MCHP','COF','MSI','GWW','TRV','HUM','AIG',
        'ALL','CPRT','USB','O','STZ','MPC','KMB','VRSK','AZO','FMS','NSC','NEM','GLW','PAYX','GEV','IT','KMI','KDP','WMB','BKR','WELL',
        'DOW','A','PRU','STT','OTIS','FAST','KR','CEG','ODFL','CTVA','ED','VICI','BBY','EIX','HPQ','DD','HAL','VLO','OKE','TRGP','MTD',
        'KEYS','K','FITB','EFX','EBAY','WST','WDC','ROK','GPN','FICO','ZBRA','TER','ALGN','VTR','AVB','EQR','ARE','INVH','SBAC','EXR',
        'MAA','CPT','BXP','UDR','DOC','REG','KIM','FRT','HST','STAG','NNN','WPC','ADC','PLD','SPG'
    ]))

    print(f"Analyzing simulated holdings for ETF: {etf_ticker_placeholder}")
    print(f"Number of tickers to analyze: {len(placeholder_tickers)}")

    endDate = dt.datetime.now()
    startDate = endDate - dt.timedelta(days=365*5)

    data = yf.download(
        placeholder_tickers + ['GLD'],
        start=startDate,
        end=endDate,
        auto_adjust=True,
        threads=True,
        progress=False
    )['Close']

    # Risk-free rate annual
    rf = 0.03447

        # Keep only tickers with at least 80% of historical data
    min_data_points = int(len(data) * 0.8)
    data = data.dropna(thresh=min_data_points, axis=1)
    print(f"Downloaded data for {data.shape[1]} tickers ({startDate.date()} to {endDate.date()})")

    # Daily log returns
    log_returns = np.log(data / data.shift(1)).dropna()
    print(f"Calculated daily log returns. Shape: {log_returns.shape}")

    # Annualized returns and volatility
    annual_mean_returns = log_returns.mean() * 252
    annual_std_dev = log_returns.std() * np.sqrt(252)

    # Sharpe ratios
    sharpe_ratios = (annual_mean_returns - rf) / annual_std_dev
    print(f"Calculated Sharpe ratios for {len(sharpe_ratios)} tickers.")

    # Top 15 tickers by Sharpe ratio
    top_15_actions = sharpe_ratios.drop('GLD').nlargest(15)
    # Include GLD
    top_15_pf = pd.concat([top_15_actions, pd.Series({'GLD': sharpe_ratios['GLD']})])
    print(f"Top 15 tickers + GLD for portfolio:\n{top_15_pf}")

    plt.figure(figsize=(10,6))
    top_15_pf.sort_values().plot(kind='barh', color='skyblue')
    plt.title(f"Top 15 Sharpe Ratios + Gold - {etf_ticker_placeholder}")
    plt.xlabel("Sharpe Ratio")
    plt.grid(axis='x', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()

    return top_15_pf, log_returns[top_15_pf.index]

top_15_pf, returns_pf = filtrage_ETF("Amundi Core MSCI World")

print("Tickers sélectionnés pour le portefeuille :")
print(top_15_pf.index.tolist())

print("--- Aperçu du Dataset Nettoyé (Rendements Log) ---")
print(returns_pf.head().to_markdown())

In [ ]:
cumulative_prices = (returns_pf + 1).cumprod() * 100

plt.figure(figsize=(12,6))
for ticker in cumulative_prices.columns:
    plt.plot(cumulative_prices[ticker], label=ticker)
plt.title("Évolution des prix base 100 des actifs retenus")
plt.xlabel("Date")
plt.ylabel("Prix cumulés (base 100)")
plt.legend(fontsize=8, ncol=3)
plt.grid(True)
plt.tight_layout()
plt.show()

centered_returns = returns_pf - returns_pf.mean()

plt.figure(figsize=(12,6))
for ticker in centered_returns.columns:
    plt.plot(centered_returns[ticker], alpha=0.7, label=ticker)
plt.title("Visualisation des rendements logarithmiques")
plt.xlabel("Date")
plt.ylabel("Rendement")
plt.legend(fontsize=8, ncol=3)
plt.grid(True)
plt.tight_layout()
plt.show()

import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

corr_matrix = returns_pf.corr()
heat_values = np.abs(corr_matrix)
plt.figure(figsize=(12,10))
sns.heatmap(heat_values, annot=True, fmt=".2f", cmap="Reds", cbar=True, square=True)
plt.title("Correlation Matrix")
plt.tight_layout()
plt.show()





In [ ]:
#Annualized volatility
def standard_dev(weights, cov_matrix):
  variance=weights.T @ cov_matrix @ weights #multiplying a matrix with weight ponderation
  return np.sqrt(variance)

  #Annualized return
def expected_returns(weights,log_returns):
  return np.sum(log_returns.mean()*weights)*252

  #Sharpe Ratio
def Sharpe_ratio(weights,log_returns,cov_matrix,rf):
  return (expected_returns(weights,log_returns)-rf)/standard_dev(weights,cov_matrix)

  #Value at Risk
def plot_var_distribution(returns_dollar, VaR, confidence_interval, days, title):
  plt.hist(returns_dollar.dropna(), bins=50, color='skyblue', edgecolor='black', density=True)
  plt.xlabel(f'{days}- Day Portfolio Return (Dollar Value)')
  plt.ylabel('Frequency')
  plt.title(title)
  plt.axvline(x=-VaR, color='red', linestyle='--', label=f'VaR {confidence_interval*100}%')
  plt.legend()
  plt.show()

def Value_at_risk(returns_percentage, confidence_interval, portfolio_value):

  return -np.percentile(returns_percentage, (1 - confidence_interval) * 100) * portfolio_value

def VaR_parametric_constraint_func(weights, log_returns, confidence_interval, days):

  portfolio_mean_daily_return = np.sum(log_returns.mean() * weights)

  portfolio_daily_std_dev = np.sqrt(np.dot(weights.T, np.dot(log_returns.cov(), weights)))

  alpha = 1 - confidence_interval
  z_score = norm.ppf(alpha)

  VaR_percent = -(portfolio_mean_daily_return * days + z_score * portfolio_daily_std_dev * np.sqrt(days))
  return VaR_percent

  #Max drawdown

def summarize_max_drawdown(cumulative_returns):
    running_max = cumulative_returns.expanding(min_periods=1).max()
    drawdown = (cumulative_returns - running_max) / running_max
    max_drawdown = drawdown.min()

    print(f"  Maximum Drawdown: {max_drawdown:.4f}")
    return max_drawdown

    #Calculation of Skewness & Kurtosis

def summarize_distribution_moments(optimal_returns, equal_returns):
    print("Optimal Portfolio:")
    print(f"  Skewness: {optimal_returns.skew():.4f}")
    print(f"  Kurtosis: {optimal_returns.kurt():.4f}")

    print("\nEqually Weighted Portfolio:")
    print(f"  Skewness: {equal_returns.skew():.4f}")
    print(f"  Kurtosis: {equal_returns.kurt():.4f}")

In [ ]:
def summarize_all_portfolio_analysis(tickers):
    warnings.filterwarnings('ignore', category=FutureWarning)

    endDate = dt.datetime.now()
    startDate = endDate - dt.timedelta(days=365 * 5)

    # Fixed factors, updated to match IPS
    rf = 0.03447
    confidence_interval = 0.95
    portfolio_value = 1000000
    days = 5
    num_simulations = 10000
    num_days = 252

    close_df = pd.DataFrame()
    for ticker in tickers:
        data = yf.download(ticker, start=startDate, end=endDate, progress=False, auto_adjust=False)
        close_df[ticker] = data['Adj Close']

    log_returns = np.log(close_df / close_df.shift(1)).dropna()

    cov_matrix_opt = log_returns.cov() * 252

    # Poids maximum par actif : 30 % -> upper bound 0.30
    # Poids minimum par actif : 0% -> lower bound 0
    bounds = [(0.0, 0.30) for _ in range(len(tickers))]

    # Define constraints list
    constraints = [
        {'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1}, # Sum of weights = 1

        # Volatility constraint: Annual Volatility <= 0.15
        {'type': 'ineq', 'fun': lambda weights: 0.15 - standard_dev(weights, cov_matrix_opt)},

        # Parametric VaR constraint: VaR (percentage) <= 0.10 (10%)
        {'type': 'ineq', 'fun': lambda weights: 0.10 - VaR_parametric_constraint_func(weights, log_returns, confidence_interval, days)}
    ]

    # Initial weights
    initial_weights = np.array([1 / len(tickers) for _ in range(len(tickers))])

    # Optimize the weights
    optimized_results = minimize(lambda weights: -Sharpe_ratio(weights, log_returns, cov_matrix_opt, rf),
                                 initial_weights, method='SLSQP', bounds=bounds, constraints=constraints)
    print("-- Optimal weights --")

    optimal_weights = optimized_results.x

    optimal_portfolio_returns = expected_returns(optimal_weights, log_returns)
    optimal_portfolio_std = standard_dev(optimal_weights, cov_matrix_opt)
    optimal_sharpe_ratio = Sharpe_ratio(optimal_weights, log_returns, cov_matrix_opt, rf)

    print("Optimal Weights:")
    for ticker, weight in zip(tickers, optimal_weights):
        print(f"{ticker}: {weight:.4f}")

    print(f"\nExpected Returns: {optimal_portfolio_returns:.4f}")
    print(f"Standard Deviation: {optimal_portfolio_std:.4f}")
    print(f"Sharpe Ratio: {optimal_sharpe_ratio:.4f}")

    print("\n--- IPS Constraint Checks for Optimal Portfolio ---")

    # 1. Volatility Constraint Check
    volatility_limit = 0.15
    print(f"Annual Volatility (Target max {volatility_limit*100:.0f}%) : {optimal_portfolio_std*100:.2f}%")
    if optimal_portfolio_std < volatility_limit:
        print("  Status: Respected")
    else:
        print("  Status: Violated")

    # Calculate number of assets
    num_assets = len(tickers)

    # Create an equal_weights NumPy array
    equal_weights = np.array([1/num_assets for _ in range(num_assets)])

    # Calculate expected returns for equally weighted portfolio
    equal_weighted_returns = expected_returns(equal_weights, log_returns)

    print("\n--- Expected Returns results ---")

    print(f"\nExpected Returns (Optimal Portfolio): {optimal_portfolio_returns:.4f}")
    print(f"Expected Returns (Equally Weighted Portfolio): {equal_weighted_returns:.4f}")


    print("\n--- Historical VaR Analysis ---")
    # Call summarize_historical_var
    def summarize_historical_var_updated(log_returns, optimal_weights, equal_weights, confidence_interval, portfolio_value, days):
        historical_optimal_returns = (log_returns * optimal_weights).sum(axis=1)
        range_optimal_returns = historical_optimal_returns.rolling(window=days).sum().dropna()
        range_optimal_returns_dollar = range_optimal_returns * portfolio_value

        historical_equal_returns = (log_returns * equal_weights).sum(axis=1)
        range_equal_returns = historical_equal_returns.rolling(window=days).sum().dropna()
        range_equal_returns_dollar = range_equal_returns * portfolio_value

        VaR_optimized = Value_at_risk(range_optimal_returns, confidence_interval, portfolio_value)
        VaR_equally_weighted = Value_at_risk(range_equal_returns, confidence_interval, portfolio_value)

        print(f"VaR for Optimal Portfolio: -${VaR_optimized:.2f}")
        print(f"VaR for Equally Weighted Portfolio: -${VaR_equally_weighted:.2f}")

        plot_var_distribution(range_optimal_returns_dollar, VaR_optimized, confidence_interval, days, f'Distribution of Optimized Portfolio {days}-Day Returns (Dollar Value)')
        plot_var_distribution(range_equal_returns_dollar, VaR_equally_weighted, confidence_interval, days, f'Distribution of Equally Weighted Portfolio {days}-Day Returns (Dollar Value)')

        return VaR_optimized, VaR_equally_weighted, historical_optimal_returns, historical_equal_returns

    VaR_optimal, VaR_equal, historical_optimal_returns, historical_equal_returns = \
        summarize_historical_var_updated(log_returns, optimal_weights, equal_weights, confidence_interval, portfolio_value, days)

    # 3. Value at Risk Constraint Check (Historical VaR for consistency with previous checks)
    VaR_limit_percentage = 0.10
    VaR_percentage_optimal = VaR_optimal / portfolio_value
    print(f"Value at Risk (Target max {VaR_limit_percentage*100:.0f}%) : {VaR_percentage_optimal*100:.2f}%")
    if VaR_percentage_optimal < VaR_limit_percentage:
        print("  Status: Respected")
    else:
        print("  Status: Violated")


    print("\n--- Maximum Drawdown Analysis ---")

    cumulative_optimal_returns = historical_optimal_returns.cumsum()
    print("Optimal Portfolio:")
    # Calculate max drawdown for optimal portfolio here to check constraint
    max_drawdown_optimal = summarize_max_drawdown(cumulative_optimal_returns)
    # 2. Maximum Drawdown Constraint Check
    max_drawdown_limit = -0.25
    print(f"Maximum Drawdown (Target max {max_drawdown_limit*100:.0f}%) : {max_drawdown_optimal*100:.2f}%")
    if max_drawdown_optimal > max_drawdown_limit: # Max drawdown is a negative value, so ">" means less severe loss
        print("  Status: Respected")
    else:
        print("  Status: Violated")

    cumulative_equal_returns = historical_equal_returns.cumsum()
    print("Equally Weighted Portfolio:")
    summarize_max_drawdown(cumulative_equal_returns) # This will print max drawdown for equal portfolio

    print("\n--- Distribution Moments ---")
    summarize_distribution_moments(historical_optimal_returns, historical_equal_returns)

    # --- Jarque-Bera Normality Test ---
    print("\n--- Jarque-Bera Normality Test ---")
    jb_optimal = jarque_bera(historical_optimal_returns)
    jb_equal = jarque_bera(historical_equal_returns)

    print("Optimal Portfolio Returns - Jarque-Bera Test:")
    print(f"  Statistic: {jb_optimal[0]:.4f}")
    print(f"  p-value: {jb_optimal[1]:.4f}")
    if jb_optimal[1] < 0.05: # Common significance level
        print("  Conclusion: Reject null hypothesis of normality (Not Normal)\n")
    else:
        print("  Conclusion: Fail to reject null hypothesis of normality (Normal or close to Normal)\n")

    print("Equally Weighted Portfolio Returns - Jarque-Bera Test:")
    print(f"  Statistic: {jb_equal[0]:.4f}")
    print(f"  p-value: {jb_equal[1]:.4f}")
    if jb_equal[1] < 0.05:
        print("  Conclusion: Reject null hypothesis of normality (Not Normal)\n")
    else:
        print("  Conclusion: Fail to reject null hypothesis of normality (Normal or close to Normal)\n")

    # --- Benchmark Portfolio Calculation ---
    print("\n--- Benchmark Portfolio Analysis ---")
    benchmark_tickers = ['IWDA.AS', 'GLD']
    benchmark_weights = {'IWDA.AS': 0.8, 'GLD': 0.2}

    benchmark_close_df = pd.DataFrame()
    for ticker in benchmark_tickers:
        data = yf.download(ticker, start=startDate, end=endDate, progress=False, auto_adjust=False)
        benchmark_close_df[ticker] = data['Adj Close']

    benchmark_log_returns = np.log(benchmark_close_df / benchmark_close_df.shift(1)).dropna()

    # Align benchmark returns with the existing log_returns index
    common_index = log_returns.index.intersection(benchmark_log_returns.index)
    aligned_benchmark_log_returns = benchmark_log_returns.reindex(common_index)

    # Calculate historical weighted returns for the benchmark
    historical_benchmark_returns = (
        aligned_benchmark_log_returns['IWDA.AS'] * benchmark_weights['IWDA.AS']
    ) + (
        aligned_benchmark_log_returns['GLD'] * benchmark_weights['GLD']
    )

    cumulative_benchmark_returns = historical_benchmark_returns.cumsum()
    print(f"Cumulative returns for Benchmark Portfolio (80% MSCI World, 20% Gold):\n{cumulative_benchmark_returns.iloc[-1]:.4f}")

    # Recalculate benchmark portfolio std for efficient frontier plot
    # Create a NumPy array for benchmark weights in the correct order based on aligned_benchmark_log_returns columns
    benchmark_weights_array = np.array([benchmark_weights[t] for t in aligned_benchmark_log_returns.columns])
    benchmark_portfolio_returns = expected_returns(benchmark_weights_array, aligned_benchmark_log_returns)
    benchmark_portfolio_std = standard_dev(benchmark_weights_array, aligned_benchmark_log_returns.cov() * 252)


    # --- Plotting Cumulative Returns ---
    plt.figure(figsize=(12, 6))
    plt.plot(cumulative_optimal_returns.index, cumulative_optimal_returns, label='Optimal Portfolio Cumulative Returns', color='blue')
    plt.plot(cumulative_equal_returns.index, cumulative_equal_returns, label='Equally Weighted Portfolio Cumulative Returns', color='green')
    plt.plot(cumulative_benchmark_returns.index, cumulative_benchmark_returns, label='Benchmark Portfolio Cumulative Returns (80% IWDA.AS, 20% GLD)', color='red')
    plt.title('Cumulative Returns Comparison')
    plt.xlabel('Date')
    plt.ylabel('Cumulative Log Returns')
    plt.legend()
    plt.grid(True)
    plt.show()

    print("\n--- Efficient Frontier Analysis ---")

    # Define function for simulating random portfolios
    def simulate_random_portfolios(num_portfolios, log_returns, cov_matrix_opt, rf):
        portfolio_returns = []
        portfolio_volatilities = []
        portfolio_sharpe_ratios = []

        num_assets = len(log_returns.columns)

        for _ in range(num_portfolios):
            weights = np.random.random(num_assets)
            weights /= np.sum(weights) # Normalize weights to sum to 1

            # Implement allocation constraints: max weight 0.30 per asset
            # Regenerate if any weight exceeds 0.30
            # This is a simple approach, more sophisticated methods could be used
            while np.any(weights > 0.30):
                weights = np.random.random(num_assets)
                weights /= np.sum(weights)

            # Calculate portfolio metrics
            ret = expected_returns(weights, log_returns)
            vol = standard_dev(weights, cov_matrix_opt)
            sharpe = Sharpe_ratio(weights, log_returns, cov_matrix_opt, rf)

            portfolio_returns.append(ret)
            portfolio_volatilities.append(vol)
            portfolio_sharpe_ratios.append(sharpe)

        return np.array(portfolio_returns), np.array(portfolio_volatilities), np.array(portfolio_sharpe_ratios)

    # Define function for plotting the efficient frontier
    def plot_efficient_frontier(random_volatilities, random_returns, random_sharpe_ratios,
                               optimal_volatility, optimal_return,
                               benchmark_volatility, benchmark_return):
        plt.figure(figsize=(12, 8))
        # Plot random portfolios, colored by Sharpe Ratio
        scatter = plt.scatter(random_volatilities, random_returns, c=random_sharpe_ratios, cmap='viridis', s=10, alpha=0.6, label='Random Portfolios')
        plt.colorbar(scatter, label='Sharpe Ratio')

        # Plot optimal portfolio
        plt.scatter(optimal_volatility, optimal_return, color='red', marker='*', s=300, label='Optimal Portfolio (Max Sharpe Ratio)')

        # Plot benchmark portfolio
        plt.scatter(benchmark_volatility, benchmark_return, color='purple', marker='X', s=200, label='Benchmark Portfolio')

        plt.title('Efficient Frontier with Random, Optimal, and Benchmark Portfolios')
        plt.xlabel('Annualized Volatility (Standard Deviation)')
        plt.ylabel('Annualized Expected Return')
        plt.grid(True)
        plt.legend()
        plt.tight_layout()
        plt.show()

    num_portfolios = 10000
    random_port_returns, random_port_volatilities, random_port_sharpe_ratios = simulate_random_portfolios(
        num_portfolios, log_returns, cov_matrix_opt, rf
    )

    plot_efficient_frontier(random_port_volatilities, random_port_returns, random_port_sharpe_ratios,
                            optimal_portfolio_std, optimal_portfolio_returns,
                            benchmark_portfolio_std, benchmark_portfolio_returns)

    print("\n--- Monte Carlo Simulation (Bootstrap) ---")

    # Get number of historical observations
    n_historical_days = len(log_returns)

    plt.figure(figsize=(15, 7))

    # --- Optimal Portfolio Simulation ---
    simulated_optimal_portfolio_values = np.zeros((num_days, num_simulations))
    for s in range(num_simulations):
        # Bootstrap: randomly select num_days rows from historical log_returns with replacement
        random_indices = np.random.choice(n_historical_days, num_days, replace=True)
        bootstrapped_daily_returns = log_returns.iloc[random_indices]

        # Calculate portfolio daily returns using bootstrapped returns and optimal weights
        portfolio_daily_returns_optimal = np.dot(bootstrapped_daily_returns, optimal_weights)
        simulated_optimal_portfolio_values[:, s] = portfolio_value * np.exp(np.cumsum(portfolio_daily_returns_optimal))

    plt.subplot(1, 2, 1)
    plt.plot(simulated_optimal_portfolio_values)
    plt.title(f'Optimal Portfolio Monte Carlo Simulation (Bootstrap, {num_simulations} runs)')
    plt.xlabel('Days')
    plt.ylabel('Portfolio Value')
    plt.grid(True)

    # --- Equally Weighted Portfolio Simulation ---
    simulated_equal_portfolio_values = np.zeros((num_days, num_simulations))
    for s in range(num_simulations):
        # Bootstrap: randomly select num_days rows from historical log_returns with replacement
        random_indices = np.random.choice(n_historical_days, num_days, replace=True)
        bootstrapped_daily_returns = log_returns.iloc[random_indices]

        # Calculate portfolio daily returns using bootstrapped returns and equal weights
        portfolio_daily_returns_equal = np.dot(bootstrapped_daily_returns, equal_weights)
        simulated_equal_portfolio_values[:, s] = portfolio_value * np.exp(np.cumsum(portfolio_daily_returns_equal))

    plt.subplot(1, 2, 2)
    plt.plot(simulated_equal_portfolio_values)
    plt.title(f'Equally Weighted Portfolio Monte Carlo Simulation (Bootstrap, {num_simulations} runs)')
    plt.xlabel('Days')
    plt.ylabel('Portfolio Value')
    plt.grid(True)

    plt.tight_layout()
    plt.show()


    # Summary Statistics for Final Portfolio Values
    final_optimal_values = simulated_optimal_portfolio_values[-1, :]
    final_equal_values = simulated_equal_portfolio_values[-1, :]

    print("\n--- Summary Statistics for Final Portfolio Values ---")
    print("Optimal Portfolio:")
    print(f"  Mean: ${np.mean(final_optimal_values):.2f}")
    print(f"  Median: ${np.median(final_optimal_values):.2f}")
    print(f"  Standard Deviation: ${np.std(final_optimal_values):.2f}")
    print(f"  Minimum: ${np.min(final_optimal_values):.2f}")
    print(f"  Maximum: ${np.max(final_optimal_values):.2f}")

    print("\nEqually Weighted Portfolio:")
    print(f"  Mean: ${np.mean(final_equal_values):.2f}")
    print(f"  Median: ${np.median(final_equal_values):.2f}")
    print(f"  Standard Deviation: ${np.std(final_equal_values):.2f}")
    print(f"  Minimum: ${np.min(final_equal_values):.2f}")
    print(f"  Maximum: ${np.max(final_equal_values):.2f}")

    print("\n--- Comprehensive Portfolio Analysis Complete ---")




In [ ]:
tickers_for_analysis = top_15_pf.index.tolist()
summarize_all_portfolio_analysis(tickers_for_analysis)

In [ ]:
import matplotlib.pyplot as plt

def plot_optimal_pie(tickers, optimal_weights):
    plt.figure(figsize=(8,8))
    plt.pie(optimal_weights, labels=tickers, autopct='%1.1f%%', startangle=90,
            colors=plt.cm.tab20.colors)
    plt.title("Optimal Portfolio Allocation")
    plt.axis('equal')
    plt.show()

plot_optimal_pie(tickers, optimal_weights)


In [ ]:
import seaborn as sns

# Calcul de la matrice de corrélation
corr_matrix = returns_pf.corr()

# Affichage de la corrélation de l'Or avec les autres actifs
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix[['GLD']].sort_values(by='GLD', ascending=False),
            annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title("Corrélation de l'or avec les actifs du portefeuille")
plt.show()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

tickers = ['GE', 'MCK', 'AVGO', 'CEG', 'APH', 'LLY', 'IBM', 'PLTR',
           'WMT', 'TRGP', 'NVDA', 'TJX', 'GS', 'WELL', 'WMB', 'GLD']

optimal_weights = np.array([0.0930, 0.2459, 0.0477, 0.0120, 0.0, 0.0945, 0.0036, 0.0129,
                            0.1285, 0.0285, 0.0039, 0.0, 0.0, 0.0209, 0.0087, 0.3000])

equal_weights = np.array([1/len(tickers) for _ in tickers])

days = 10
np.random.seed(42)
cov_matrix = log_returns.cov().values

stress_cov = cov_matrix.copy()


stress_cov[:-1, :-1] *= 3


mu_shock = np.array([-0.025]*(len(tickers)-1) + [0.002])
# -2.5% / jour actions (~ -25% en 10j)

shock_returns = np.random.multivariate_normal(
    mean=mu_shock,
    cov=stress_cov,
    size=days
)

log_returns_shock = pd.DataFrame(shock_returns, columns=tickers)

def cumulative_portfolio_returns(weights, returns):
    return (returns @ weights).cumsum()

cum_optimal = cumulative_portfolio_returns(optimal_weights, log_returns_shock)
cum_equal = cumulative_portfolio_returns(equal_weights, log_returns_shock)
cum_gld = log_returns_shock['GLD'].cumsum()

plt.figure(figsize=(10,6))
plt.plot(cum_optimal, label='Optimal Portfolio', linewidth=2)
plt.plot(cum_equal, label='Equally Weighted Portfolio', linewidth=2)
plt.plot(cum_gld, label='GLD Benchmark', linestyle='--', linewidth=2)

plt.title('Stress Test Crash 2008-like')
plt.xlabel('Jours')
plt.ylabel('Rendements cumulés (log)')
plt.legend()
plt.grid(True)
plt.show()

def portfolio_stats(cum_returns, name):
    total_return = cum_returns.iloc[-1]
    max_drawdown = (cum_returns - cum_returns.cummax()).min()
    vol = cum_returns.diff().std() * np.sqrt(252)

    print(f"{name}")
    print(f"Total Return: {total_return:.4f}")
    print(f"Max Drawdown: {max_drawdown:.4f}")
    print(f"Volatility: {vol:.4f}")
    print("")

print("=== Résultats Stress Test Crash 2008-like ===")
portfolio_stats(cum_optimal, "Optimal Portfolio")
portfolio_stats(cum_equal, "Equally Weighted Portfolio")
portfolio_stats(cum_gld, "GLD Benchmark")


In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from scipy.optimize import minimize


tickers = ['GE', 'MCK', 'AVGO', 'CEG', 'APH', 'LLY', 'IBM', 'PLTR',
           'WMT', 'TRGP', 'NVDA', 'TJX', 'GS', 'WELL', 'WMB', 'GLD']

optimal_weights_before = np.array([0.0930, 0.2459, 0.0477, 0.0120, 0.0, 0.0945, 0.0036, 0.0129,
                                   0.1285, 0.0285, 0.0039, 0.0, 0.0, 0.0209, 0.0087, 0.3000])

# Téléchargement des données
endDate = pd.Timestamp.today()
startDate = endDate - pd.Timedelta(days=365*5)
data = yf.download(tickers, start=startDate, end=endDate, auto_adjust=True, progress=False)['Close']
log_returns = np.log(data / data.shift(1)).dropna()


window = 42  # 2 mois
portfolio_returns = log_returns.dot(optimal_weights_before)

features = pd.DataFrame(index=log_returns.index)
features['MA'] = portfolio_returns.rolling(window).mean()
features['Volatility'] = portfolio_returns.rolling(window).std()
features['NegativeRatio'] = (portfolio_returns.rolling(window).apply(lambda x: (x<0).sum())/window)
features['Market'] = np.where(features['NegativeRatio']>0.5, 0, 1)  # 0 = baissier, 1 = haussier
features = features.dropna()

X = features[['MA','Volatility','NegativeRatio']]
y = features['Market']
X_scaled = StandardScaler().fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, shuffle=False)


rf = RandomForestClassifier(n_estimators=200, random_state=42)
rf.fit(X_train, y_train)
signal = rf.predict(X_scaled[-1].reshape(1,-1))[0]
market_condition = 'Baissier' if signal==0 else 'Haussier'
print(f"Signal ML détecté : {market_condition}")


cov_matrix = log_returns.cov() * 252
expected_returns = log_returns.mean() * 252
rf_rate = 0.03447  # taux sans risque

def markowitz_sharpe(weights, expected_returns, cov_matrix, rf=rf_rate):
    port_return = np.dot(weights, expected_returns)
    port_vol = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    sharpe = (port_return - rf) / port_vol
    return -sharpe

constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x)-1})
bounds = [(0,0.3) for _ in tickers]


init_weights = optimal_weights_before.copy()
if market_condition=='Baissier':
    gl_index = tickers.index('GLD')
    init_weights[gl_index] += 0.1
    other_indices = [i for i in range(len(tickers)) if i != gl_index]
    init_weights[other_indices] *= (1 - init_weights[gl_index])/sum(init_weights[other_indices])

res = minimize(markowitz_sharpe, init_weights, args=(expected_returns.values, cov_matrix.values),
               method='SLSQP', bounds=bounds, constraints=constraints)
adjusted_weights = res.x


def portfolio_metrics(weights, log_returns, rf=rf_rate):
    port_ret = np.sum(log_returns.mean() * weights) * 252
    port_vol = np.sqrt(np.dot(weights.T, np.dot(log_returns.cov()*252, weights)))
    sharpe = (port_ret - rf)/port_vol
    return port_ret, port_vol, sharpe

ret_before, vol_before, sharpe_before = portfolio_metrics(optimal_weights_before, log_returns)
ret_after, vol_after, sharpe_after = portfolio_metrics(adjusted_weights, log_returns)

df_metrics = pd.DataFrame({
    'Metric': ['Expected Return','Volatility','Sharpe Ratio'],
    'Before ML+Markowitz':[ret_before, vol_before, sharpe_before],
    'After ML+Markowitz':[ret_after, vol_after, sharpe_after]
})
print("\n--- Rendements & Risques Avant/Après ---")
print(df_metrics)


df_weights = pd.DataFrame({
    'Ticker': tickers,
    'Before ML': optimal_weights_before,
    'After ML': adjusted_weights
})

plt.figure(figsize=(12,6))
bar_width = 0.35
x = np.arange(len(tickers))
plt.bar(x - bar_width/2, optimal_weights_before, width=bar_width, label='Before ML', color='skyblue')
plt.bar(x + bar_width/2, adjusted_weights, width=bar_width, label='After ML', color='orange')
plt.xticks(x, tickers, rotation=45, ha='right')
plt.ylabel("Poids dans le portefeuille")
plt.title("Portefeuille Avant / Après ML")
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()
